In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 라이브러리 설치

In [1]:
!pip3 install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.8/288.8 kB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 14.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mdit-py-plugins to determine which version is compatible with other requirements. This could take a while.
 

# gradio 구현(1) : Hello!

In [ ]:
import gradio as gr

def greet(name):
    return "Hello " + name + "!"

demo = gr.Interface(fn=greet,
                    allow_flagging='manual',
                    inputs = "text",
                    outputs = "text")

demo.launch(server_name = "0.0.0.0",
            server_port = 700,
            # auth = ("noname", "passwdl"),
            share=True
            )

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://90ed355a55a23d122a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


- launch()에서 share=True로 지정하면 gradio에서 제공하는 소스를 사용하게 된다.

# gradio 구현(2) : 분류 모델

In [ ]:
import tensorflow as tf

inception_net = tf.keras.applications.MobileNetV2()

14536120/14536120 [==============================] - 0s 0us/step


In [ ]:
import requests

response = requests.get("https://git.io/JJkYN")
labels = response.text.split("\n")

def classify_image(inp):
    inp = inp.reshape((-1, 224, 224, 3))
    inp = tf.keras.applications.mobilenet_v2.preprocess_input(inp)
    prediction = inception_net.predict(inp).flatten()
    confidences = {labels[i]: float(prediction[i]) for i in range(1000)}
    return confidences

In [ ]:
import gradio as gr

classification = gr.Interface(fn=classify_image,
                              inputs = gr.Image(shape=(224,224)),
                              outputs = gr.Label(num_top_classes=3))

classification.launch(share=True,
                      server_name = "0.0.0.0",
                      server_port = 600
                      # auth = ("noname", "passwdl"),
                      )

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://9297d34693bc8cc930.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


# 1. 퍼스널 컬러 앱 구현

## cifar10으로 시험 코드

In [ ]:
# knn / zero shot의 경우

## 1. 데이터 수집(라벨링 된)
## 2. 데이터 입력
## 3. 모델이 웜/쿨인지 결과 출력

In [ ]:
# cifar10으로 예상 모형 만들기

import gradio as gr
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

def classify_image(image):
    # Preprocess the image
    image = image.reshape((1, 32 * 32 * 3)) / 255.0

    # Load the trained model
    model = keras.models.load_model('keras_cifar10_model.h5')

    # Perform prediction
    prediction = model.predict(image)
    class_index = np.argmax(prediction)
    class_name = class_names[class_index]

    return class_name

# Load CIFAR-10 dataset and split into train, validation, and test sets
(x_train_full, y_train_full), (x_test, y_test) = cifar10.load_data()
x_train, x_val, y_train, y_val = train_test_split(x_train_full,
                                                  y_train_full,
                                                  test_size=0.25,
                                                  random_state=42)

class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

# Preprocess the data
x_train = x_train.reshape(x_train.shape[0], 32 * 32 * 3)
x_val = x_val.reshape(x_val.shape[0], 32 * 32 * 3)
x_test = x_test.reshape(x_test.shape[0], 32 * 32 * 3)

x_train = x_train / 255.
x_val = x_val / 255.
x_test = x_test / 255.

y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

# Define the model architecture
model = keras.models.Sequential()
model.add(layers.Input(shape=(3072,)))
model.add(layers.Dense(2048, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax', name='output'))

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Define callbacks
check_point_cb = callbacks.ModelCheckpoint('keras_cifar10_model.h5', save_best_only=True)
early_stopping_cb = callbacks.EarlyStopping(patience=3,
                                            monitor='val_loss',
                                            restore_best_weights=True)

# Train the model
history = model.fit(x_train,
                    y_train,
                    epochs=40,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    callbacks=[check_point_cb, early_stopping_cb])

170498071/170498071 [==============================] - 2s 0us/step
Epoch 1/40
74/74 [==============================] - 36s 473ms/step - loss: 2.4578 - accuracy: 0.2354 - val_loss: 6.6257 - val_accuracy: 0.1426
Epoch 2/40
74/74 [==============================] - 35s 468ms/step - loss: 2.0762 - accuracy: 0.3089 - val_loss: 3.5605 - val_accuracy: 0.2335
Epoch 3/40
74/74 [==============================] - 35s 471ms/step - loss: 1.9017 - accuracy: 0.3467 - val_loss: 1.9820 - val_accuracy: 0.3050
Epoch 4/40
74/74 [==============================] - 35s 470ms/step - loss: 1.7887 - accuracy: 0.3739 - val_loss: 1.8091 - val_accuracy: 0.3538
Epoch 5/40
74/74 [==============================] - 32s 439ms/step - loss: 1.7203 - accuracy: 0.3960 - val_loss: 1.7812 - val_accuracy: 0.3682
Epoch 6/40
74/74 [==============================] - 34s 454ms/step - loss: 1.6731 - accuracy: 0.4113 - val_loss: 1.7431 - val_accuracy: 0.3920
Epoch 7/40
74/74 [==============================] - 33s 444ms/step - loss: 

In [ ]:
# Define the Gradio interface
iface = gr.Interface(fn=classify_image,
                     inputs=gr.inputs.Image(shape=(32, 32)),
                     outputs=gr.outputs.Label(num_top_classes=2))

# Launch the interface
iface.launch(share=True,
             server_name = "0.0.0.0",
             server_port = 300)
            #  auth = ("noname", "passwdl"))

<ipython-input-8-efccfe3798c5>:3: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  inputs=gr.inputs.Image(shape=(32, 32)),
<ipython-input-8-efccfe3798c5>:3: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  inputs=gr.inputs.Image(shape=(32, 32)),
<ipython-input-8-efccfe3798c5>:4: GradioDeprecationWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  outputs=gr.outputs.Label(num_top_classes=2))
<ipython-input-8-efccfe3798c5>:4: GradioUnusedKwargWarning: You have unused kwarg parameters in Label, please remove them: {'type': 'auto'}
  outputs=gr.outputs.Label(num_top_classes=2))


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://0950c8587530685a44.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


- 위에서는 cifar10의 딥러닝 학습 분류 모델을 이용한 것을 gradio로 구현한 코드이다.
    퍼스널 컬러 분류 코드가 만들어지면 위와 같이 구현할 수 있을 것이다.

- 퍼스널 컬러의(구현한다면) 퍼센테이지(%)를 확인할 수 있게 하기 위해 label의 top2를 나타내게 하고 싶은데 데모에선 됐는데 위에선 표시가 안 됐다. 다시 코드를 살펴보고 수정해야 할 것 같음

## KNN 적용 퍼컬 앱 구현(1)

In [ ]:
# 라이브러리 블러오기

import tensorflow as tf
import numpy as np
import glob
from PIL import Image

In [ ]:
# 데이터 불러오기

def load_images_from_directory(directory_path):
    image_paths = glob.glob(directory_path + '/*.jpg')  # 디렉토리 내의 모든 jpg 파일 경로 찾기
    image_paths.extend(glob.glob(directory_path + '/*.jpeg'))

    images = []
    for image_path in image_paths:
        image = Image.open(image_path)
        image_np = np.array(image)
        images.append(image_np)

    return images

In [ ]:
# %cd /content/drive/MyDrive/Colab Notebooks/AIFFEL/Data/four_seasons/웜

# !unzip -qq "/content/drive/MyDrive/Colab Notebooks/AIFFEL/Four_seasons/사계절_연예인 이미지 데이터셋/prototype/웜톤/웜톤_jpg.zip"

In [ ]:
# %cd /content/drive/MyDrive/Colab Notebooks/AIFFEL/Data/four_seasons/쿨

# !unzip -qq "/content/drive/MyDrive/Colab Notebooks/AIFFEL/Four_seasons/사계절_연예인 이미지 데이터셋/prototype/쿨톤/쿨톤_jpg.zip"

In [ ]:
# 모든 이미지 불러오기
directory_path = "/content/drive/MyDrive/Colab Notebooks/AIFFEL/Data/four_seasons/쿨"  # 디렉토리 경로
directory_path2 = "/content/drive/MyDrive/Colab Notebooks/AIFFEL/Data/four_seasons/웜"  # 디렉토리 경로
images = load_images_from_directory(directory_path)
images2 = load_images_from_directory(directory_path2)

length = len(images)
print(length)
print(len(images2))

50
50


In [ ]:
# 이미지 리스트 합치기

images = np.concatenate((images, images2), axis=0)
print(len(images))

100


<__array_function__ internals>:180: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [ ]:
# 데이터 전처리

import numpy as np
import cv2

# 새로운 크기로 리사이즈할 이미지의 너비와 높이
new_width = 200
new_height = 200

resized_images = []
for image in images:
    resized_image = cv2.resize(image, (new_width, new_height))
    resized_images.append(resized_image)

In [ ]:
# resized_images 리스트를 NumPy 배열로 변환
resized_images_array = np.array(resized_images)

# 변환된 NumPy 배열 출력
print(resized_images_array.shape)

(100, 200, 200, 3)


In [ ]:
images = resized_images_array.reshape(resized_images_array.shape[0], 200 * 200 * 3)

In [ ]:
# labels에 대한 코드
num = len(images)
labels = np.zeros(num)

# 웜, 쿨의 데이터를 0과 1로 나눠 표시되도록
labels[length:] = 1

# 0이 쿨톤 1이 웜톤
print(len(images))
print(labels)

100
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1.]


In [ ]:
train_ratio = 0.7  # 훈련 세트 비율
val_ratio = 0.2    # 검증 세트 비율
train_data, train_labels, val_data, val_labels, test_data, test_labels = split_data(images, labels, train_ratio, val_ratio)
print("train: ", train_data.shape)
print("val: ", val_data.shape)
print("test: ", test_data.shape)

총 데이터 갯수:  100
총 라벨 갯수:  100
train:  (70, 120000)
val:  (20, 120000)
test:  (10, 120000)


- 여기서 라벨 개수가 2개가 나와야하는거 아닌가?

In [ ]:
# 검증 셋에서 가장 잘 동작하는 hyperparameter 들을 찾는다.
validation_accuracies = []
for k in [1, 3, 5, 10, 20, 25, 30]:

    # 특정 k 값을 정해서 검증 데이터에 대해 평가할 때 사용한다.
    nn = NearestNeighbor()
    nn.train(train_data, train_labels)
    # 여기서는 k를 input으로 받을 수 있도록 변형된 NearestNeighbor 클래스가 있다고 가정하자.
    Yval_predict = nn.predict(val_data, k = k)
    acc = np.mean(Yval_predict == val_labels)
    print('accuracy: %f' % (acc,))

    # 검증 셋에 대한 정확도를 저장해 놓는다.
    validation_accuracies.append((k, acc))

accuracy: 0.600000
accuracy: 0.550000
accuracy: 0.400000
accuracy: 0.400000
accuracy: 0.600000
accuracy: 0.500000
accuracy: 0.550000


In [ ]:
class_names = ['웜톤', '쿨톤']

# def personal_color(ima):
  # img =

def classify_image(image):
    # Preprocess the image
    image = image.reshape((1, 32 * 32 * 3)) / 255.0

    # Load the trained model
    model = keras.models.load_model('keras_cifar10_model.h5')

    # Perform prediction
    prediction = model.predict(image)
    class_index = np.argmax(prediction)
    class_name = class_names[class_index]

    return class_name

In [ ]:
# KNN 퍼스널 컬러 분류 함수 만들기

def personal_color(img):
    nn = NearestNeighbor()
    X_train = train_data  # Training data features
    y_train = train_labels  # Training data labels
    nn.train(X_train, y_train)

    prediction = nn.predict(img, k = 5)
    return prediction

In [ ]:
# gradio로 시각화? 웹 구현
import gradio as gr

classification = gr.Interface(fn=personal_color,
                              # inputs="image",
                              # outputs="label",
                              inputs=gr.inputs.Image(shape=(200, 200)),
                              outputs=gr.outputs.Label(num_top_classes=2))


classification.launch(share=True,
                      server_name = "0.0.0.0",
                      server_port = 8081)
                      # auth = ("noname", "passwdl"))

<ipython-input-27-73227275436d>:7: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  inputs=gr.inputs.Image(shape=(200, 200)),
<ipython-input-27-73227275436d>:7: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  inputs=gr.inputs.Image(shape=(200, 200)),
<ipython-input-27-73227275436d>:8: GradioDeprecationWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  outputs=gr.outputs.Label(num_top_classes=2))
<ipython-input-27-73227275436d>:8: GradioUnusedKwargWarning: You have unused kwarg parameters in Label, please remove them: {'type': 'auto'}
  outputs=gr.outputs.Label(num_top_classes=2))


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://5b5480191a0b24c172.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
def personal_color(img):
    # Preprocess the new image if needed

    # Load the pre-trained model or create a new instance of NearestNeighbor
    nn = NearestNeighbor()

    # Load the training data
    X_train = train_data  # Training data features
    y_train = train_labels  # Training data labels

    # Train the model
    nn.train(X_train, y_train)

    # Convert the new image to the appropriate format (e.g., N x D)
    X_test = img # Preprocess the new image to match the training data format

    # Set the value of k for k-nearest neighbors
    k = 5  # Define the value of k

    # Predict the label of the new image
    predicted_label = nn.predict(X_test, k)

    # Return the predicted label
    return predicted_label


In [ ]:
img_1 = '/content/drive/MyDrive/Colab Notebooks/AIFFEL/Data/sticker_app/images/서현진1.jpg'
personal_color(img_1) ## 시도해보았으나 에러뜸. 함수를  같다.

In [ ]:
import gradio as gr

classification = gr.Interface(fn=personal_color,
                              inputs=gr.inputs.Image(shape=(200, 200)),
                              outputs=gr.outputs.Label(num_top_classes=2))

<ipython-input-30-444224c19c17>:4: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  inputs=gr.inputs.Image(shape=(200, 200)),
<ipython-input-30-444224c19c17>:4: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  inputs=gr.inputs.Image(shape=(200, 200)),
<ipython-input-30-444224c19c17>:5: GradioDeprecationWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  outputs=gr.outputs.Label(num_top_classes=2))
<ipython-input-30-444224c19c17>:5: GradioUnusedKwargWarning: You have unused kwarg parameters in Label, please remove them: {'type': 'auto'}
  outputs=gr.outputs.Label(num_top_classes=2))


In [ ]:
# classification.launch(share=True)

## KNN : Skin detection을 적용한 앱 구현(2)

In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import os
import glob
from PIL import Image

In [ ]:
def Skin_Detection(img_path):

  #Open a simple image
  img=cv2.imread(img_path)

  #converting from gbr to YCbCr color space
  img_YCrCb = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
  #skin color range for hsv color space
  YCrCb_mask = cv2.inRange(img_YCrCb, (0, 135, 85), (255,180,135))
  YCrCb_mask = cv2.morphologyEx(YCrCb_mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))

  YCrCb_result = cv2.bitwise_not(YCrCb_mask)
  YCrCb_skin_detected = cv2.bitwise_and(img, img, mask=YCrCb_mask)  # img와 YCrCb_mask를 입력으로 사용하여 원본 이미지에서 피부 픽셀만 추출

  return img_YCrCb, YCrCb_mask

In [ ]:
# skin 좌표 추출, YCrCb 픽셀값 추출
def coordinate_extraction(mask, img):
  skin = np.argwhere(mask == 255) # mask의 픽셀값이 255인 skin부분의 좌표 추출 : (rgb로 따지는 거야? ybbcr말고?)

  pixel_values = [] # YCbCr 픽셀값 저장을 위한 리스트

  for index in skin:
    y, x = index  #일반적으로 생각하는 가로 x, 세로 y로 표현하기 위해 위치 바꿈
    #print(f"x: {x}, y: {y}") # skin 범위 좌표값 출력(스압주의)
    pixel_value = img[y, x]
    pixel_values.append(pixel_value)

  for coord, pixel_value in zip(skin, pixel_values): # skin부분의 좌표값과 픽셀값 출력을 위한 함수
    x, y = coord
    # print(f"Coordinate: ({x}, {y}), Pixel Value: {pixel_value}")

  return pixel_values

In [ ]:
# 픽셀의 평균값 구하기
def pixel_means(pixel_values):
  # 열 별로 원소들을 더하기
  column_sums = np.sum(pixel_values, axis=0)

  # pixel_values를 NumPy 배열로 변환
  pixel_values = np.array(pixel_values)

  # 열 별로 평균 계산하기
  column_means = column_sums / pixel_values.shape[0]

  return column_means

In [ ]:
def get_dict(dir_path):
  file_list = os.listdir(dir_path)

  img_list = []
  result_list = []

  for file in file_list:
    img_path = os.path.join(dir_path, file)
    if os.path.isdir(img_path):  # 만약 디렉토리인 경우, 무시하고 다음 파일로 넘어감
      continue
    file_name = file
    file_name = os.path.splitext(file_name)[0] # 파일 이름과 확장자 분리
    img_YCrCb,YCrCb_mask = Skin_Detection(img_path)
    img_list.append([img_YCrCb, YCrCb_mask]) # 그림 show()를 위한 용도

    # YCrCb
    YCrCb_pixel_values = coordinate_extraction(YCrCb_mask, img_YCrCb)
    YCrCb_pixel_means = pixel_means(YCrCb_pixel_values)
    #print(YCrCb_pixel_means)

    result_list.append(YCrCb_pixel_means)

  return result_list

In [ ]:
path = "/content/drive/MyDrive/Colab Notebooks/AIFFEL/Data/four_seasons/쿨"  # 디렉토리 경로
path2 = "/content/drive/MyDrive/Colab Notebooks/AIFFEL/Data/four_seasons/웜"  # 디렉토리 경로

In [ ]:
cool_result  = get_dict(path) # 1분

In [ ]:
warm_result  = get_dict(path2) # 1분

In [ ]:
images = np.concatenate((cool_result, warm_result), axis=0)
print(images) #스압 조큼 있음

In [ ]:
# labels에 대한 코드
num = len(images)
labels = np.zeros(num)

# 처음 50개의 요소를 1로 변경
labels[50:] = 1

# 0이 쿨톤 1이 웜톤
print(len(images))
print(labels)

100
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1.]


In [ ]:
# train, val, test 분리

def split_data(data, labels, train_ratio, val_ratio):
    # 데이터 개수 확인
    num_examples = len(data)
    num_lables = len(labels)
    print("총 데이터 갯수: ", len(data))
    print("총 라벨 갯수: ", len(labels))

    # 인덱스를 섞은 배열 생성
    shuffled_indices = np.random.permutation(num)

    # 훈련 세트, 검증 세트, 테스트 세트 크기 계산
    train_size = int(num_examples * train_ratio)
    val_size = int(num_examples * val_ratio)
    test_size = num_examples - train_size - val_size

    # 데이터와 레이블을 분할하여 저장할 변수 초기화
    train_data = np.zeros((train_size, *data.shape[1:]))
    train_labels = np.zeros((train_size, *labels.shape[1:]))
    val_data = np.zeros((val_size, *data.shape[1:]))
    val_labels = np.zeros((val_size, *labels.shape[1:]))
    test_data = np.zeros((test_size, *data.shape[1:]))
    test_labels = np.zeros((test_size, *labels.shape[1:]))

    # 데이터와 레이블을 섞은 인덱스를 기반으로 분할
    train_indices = shuffled_indices[:train_size]
    val_indices = shuffled_indices[train_size:train_size+val_size]
    test_indices = shuffled_indices[train_size+val_size:]

    train_data = data[train_indices]
    train_labels = labels[train_indices]
    val_data = data[val_indices]
    val_labels = labels[val_indices]
    test_data = data[test_indices]
    test_labels = labels[test_indices]

    return train_data, train_labels, val_data, val_labels, test_data, test_labels

In [ ]:
train_ratio = 0.7  # 훈련 세트 비율
val_ratio = 0.2    # 검증 세트 비율
train_data, train_labels, val_data, val_labels, test_data, test_labels = split_data(images, labels, train_ratio, val_ratio)
print("train: ", train_data.shape)
print("val: ", val_data.shape)
print("test: ", test_data.shape)

총 데이터 갯수:  100
총 라벨 갯수:  100
train:  (70, 3)
val:  (20, 3)
test:  (10, 3)


In [ ]:
class NearestNeighbor(object):
    def __init__(self):
        pass

    def train(self, X, y):
        """ X is N x D where each row is an example. Y is 1-dimension of size N """
        # nearest neighbor 분류기는 단순히 모든 학습 데이터를 기억해둔다.
        self.Xtr = X
        self.ytr = y

    def predict(self, X, k):
        """ X is N x D where each row is an example we wish to predict label for """
        num_test = X.shape[0]
        # 출력 type과 입력 type이 같게 되도록 확인해준다.
        Ypred = np.zeros(num_test, dtype = self.ytr.dtype)

    # loop over all test rows
        for i in range(num_test):
        # i번째 테스트 이미지와 가장 가까운 학습 이미지를
        # L1 거리(절대값 차의 총합)를 이용하여 찾는다.
            distances = np.sum(np.abs(self.Xtr - X[i,:]), axis = 1)
            #distances = np.sqrt(np.sum(np.square(self.Xtr - X[i,:]), axis = 1))
            #min_index = np.argmin(distances) # 가장 작은 distance를 갖는 인덱스를 찾는다.
            #pred[i] = self.ytr[min_index] # 가장 가까운 이웃의 라벨로 예측

            indices = np.argsort(distances)  # 거리가 작은 순서대로 인덱스 정렬
            k_nearest_labels = self.ytr[indices[:k]]  # k개의 가장 가까운 이웃의 라벨
            unique, counts = np.unique(k_nearest_labels, return_counts=True)
            Ypred[i] = unique[np.argmax(counts)]  # 가장 많은 투표를 받은 라벨로 예측
        return Ypred

In [ ]:
# 검증 셋에서 가장 잘 동작하는 hyperparameter 들을 찾는다.
validation_accuracies = []
for k in [5, 10, 20, 25, 30]:

    # 특정 k 값을 정해서 검증 데이터에 대해 평가할 때 사용한다.
    nn = NearestNeighbor()
    nn.train(train_data, train_labels)
    # 여기서는 k를 input으로 받을 수 있도록 변형된 NearestNeighbor 클래스가 있다고 가정하자.
    Yval_predict = nn.predict(val_data, k = k)
    acc = np.mean(Yval_predict == val_labels)
    print('accuracy: %f' % (acc,))

    # 검증 셋에 대한 정확도를 저장해 놓는다.
    validation_accuracies.append((k, acc))

accuracy: 0.650000
accuracy: 0.650000
accuracy: 0.550000
accuracy: 0.450000
accuracy: 0.450000


In [ ]:
# 이제 본격적인 최고 25개 85% / 정확도 45%~85%정도 나오는듯
print(validation_accuracies)

[(5, 0.65), (10, 0.65), (20, 0.55), (25, 0.45), (30, 0.45)]


In [ ]:
print(k)

30


- k가 마지막 숫자인30으로 설정되어 있기 때문에 밑에서는 위에서 본 최고 성능을 내는 숫자로 다시 지정해주어야 한다

In [ ]:
Ytest_predict = nn.predict(test_data, k = 10)
print(Ytest_predict)
print(test_labels)

[1. 0. 1. 0. 0. 1. 1. 0. 0. 0.]
[1. 1. 0. 0. 1. 1. 0. 0. 1. 0.]


In [ ]:
def personal_color_app(img):
  Ytest_predict = nn.predict(img, k = k)
  return test_labels

In [ ]:
img = /content/drive/MyDrive/Colab Notebooks/AIFFEL/Data/four_seasons/웜/공효진.jpg
personal_color_app(img)

SyntaxError: ignored

- FAIL. 다시...

In [ ]:
import gradio as gr

classification = gr.Interface(fn=personal_color_app,
                              inputs='image',
                              outputs='label')

In [ ]:
# classification.launch(share=True)

# 2.  skin detection을 이용한 KNN 모델 및 앱 구현

1. KNN 모델 구현
2. 클래스 이름 정해서 나누기
3. 각 클래스가 반환되도록 함수 만들기
4. gradio에 연결

## 라이브러리 불러오기 및 데이터 로드

In [ ]:
# 라이브러리 불러오기

import tensorflow as tf
import numpy as np
import cv2
import os
import glob
from PIL import Image

In [ ]:
def load_images_from_directory(directory_path):
    image_paths = glob.glob(directory_path + '/*.jpg')  # 디렉토리 내의 모든 jpg 파일 경로 찾기
    #image_paths.extend(glob.glob(directory_path + '/)*.png')  # 디렉토리 내의 모든 png 파일 경로 추가 오류남
    image_paths.extend(glob.glob(directory_path + '/*.jpeg'))  # 디렉토리 내의 모든 jpeg 파일 경로 추가
    image_paths.extend(glob.glob(directory_path + '/*.jfif'))  # 디렉토리 내의 모든 jfif 파일 경로 추가

    images = []
    for image_path in image_paths:
        image = Image.open(image_path)
        image_np = np.array(image)
        images.append(image_np)

    return images

In [ ]:
# 모든 이미지 불러오기
directory_path = "/content/drive/MyDrive/Colab Notebooks/AIFFEL/Data/four_seasons/쿨"  # 디렉토리 경로
directory_path2 = "/content/drive/MyDrive/Colab Notebooks/AIFFEL/Data/four_seasons/웜"  # 디렉토리 경로
images = load_images_from_directory(directory_path)
images2 = load_images_from_directory(directory_path2)

length = len(images)
print(length)
print(len(images2))

# 이미지 리스트 합치기
images = np.concatenate((images, images2), axis=0)
print(len(images))

50
50
100


<__array_function__ internals>:180: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


## 데이터 전처리 및 train, val, test 데이터 나누기

In [ ]:
## 데이터를 train+val 데이터를 추가하기

def split_data(data, labels, train_ratio, val_ratio):
    # 데이터 개수 확인
    num_examples = len(data)
    num_lables = len(labels)
    print("총 데이터 갯수: ", len(data))
    print("총 라벨 갯수: ", len(labels))

    # 인덱스를 섞은 배열 생성
    shuffled_indices = np.random.permutation(num)

    # 훈련 세트, 검증 세트, 테스트 세트 크기 계산
    train_size = int(num_examples * train_ratio)
    val_size = int(num_examples * val_ratio)
    test_size = num_examples - train_size - val_size

    # 데이터와 레이블을 분할하여 저장할 변수 초기화
    train_data = np.zeros((train_size, *data.shape[1:]))
    train_labels = np.zeros((train_size, *labels.shape[1:]))
    val_data = np.zeros((val_size, *data.shape[1:]))
    val_labels = np.zeros((val_size, *labels.shape[1:]))
    test_data = np.zeros((test_size, *data.shape[1:]))
    test_labels = np.zeros((test_size, *labels.shape[1:]))

    # 데이터와 레이블을 섞은 인덱스를 기반으로 분할
    train_indices = shuffled_indices[:train_size]
    val_indices = shuffled_indices[train_size:train_size+val_size]
    test_indices = shuffled_indices[train_size+val_size:]

    train_data = data[train_indices]
    train_labels = labels[train_indices]
    val_data = data[val_indices]
    val_labels = labels[val_indices]
    test_data = data[test_indices]
    test_labels = labels[test_indices]
    train_full_data = np.concatente(train_data, val_data)
    train_full_labels = np.concatenate(train_labels, val_labels)

    return train_full_data, train_full_labels, train_data, train_labels, val_data, val_labels, test_data, test_labels

In [ ]:
train_ratio = 0.7  # 훈련 세트 비율
val_ratio = 0.2    # 검증 세트 비율
train_data, train_labels, val_data, val_labels, test_data, test_labels = split_data(images, labels, train_ratio, val_ratio)

print('full_train: ', train_full_data)
print("train: ", train_data.shape)
print("val: ", val_data.shape)
print("test: ", test_data.shape)

NameError: ignored

In [ ]:
class_names = ['웜톤', '쿨톤']

In [ ]:
print(train_full_labels[:2])

# Reference

- 분류 모델 공식 홈페이지 데모 버전 (https://www.gradio.app/guides/image-classification-in-tensorflow)
- 코드로 압축 풀기 (https://zeuskwon-ds.tistory.com/52)
- 모폴로지 연산 (https://bkshin.tistory.com/entry/OpenCV-19-%EB%AA%A8%ED%8F%B4%EB%A1%9C%EC%A7%80Morphology-%EC%97%B0%EC%82%B0-%EC%B9%A8%EC%8B%9D-%ED%8C%BD%EC%B0%BD-%EC%97%B4%EB%A6%BC-%EB%8B%AB%ED%9E%98-%EA%B7%B8%EB%A0%88%EB%94%94%EC%96%B8%ED%8A%B8-%ED%83%91%ED%96%87-%EB%B8%94%EB%9E%99%ED%96%87)
-